# Grid extension Examples

You can extend the grid by inheriting from the power-grid-model-ds. In this way you can profit from existing functionality but also add your own.
Here we show how to extend the Grid with output variables in the node and line arrays.

## Adding columns to the Grid arrays

Output variables are not present in the basic Grid as defined in pgm-ds, since these might or might not be useful to specific projects.
To add these as you please you can extend the definitions of the arrays in your own project.


In [1]:
import numpy as np
from numpy.typing import NDArray

from power_grid_model_ds.arrays import LineArray, NodeArray


class ExtendedNodeArray(NodeArray):
    """Extends the node array with an output value"""

    _defaults = {"u": 0}

    u: NDArray[np.float64]


class ExtendedLineArray(LineArray):
    """Extends the line array with an output value"""

    _defaults = {"i_from": 0}

    i_from: NDArray[np.float64]

## Adding the new arrays to the Grid

When these arrays have been defined they should also be provided to the Grid object.


In [2]:
from dataclasses import dataclass

from power_grid_model_ds import GraphContainer, Grid


@dataclass
class ExtendedGrid(Grid):
    """
    This is my own grid to extend.
    """

    node: ExtendedNodeArray
    line: ExtendedLineArray
    graphs: GraphContainer

## Create a grid with this array container


In [3]:
ExtendedGrid.empty()

ExtendedGrid(_id_counter=0, graphs=GraphContainer(active_graph=<power_grid_model_ds._core.model.graphs.models.rustworkx.RustworkxGraphModel object at 0x000001E33EBC27B0>, complete_graph=<power_grid_model_ds._core.model.graphs.models.rustworkx.RustworkxGraphModel object at 0x000001E33EC182D0>), node=ExtendedNodeArray([[]]), transformer=TransformerArray([[]]), three_winding_transformer=ThreeWindingTransformerArray([[]]), line=ExtendedLineArray([[]]), link=LinkArray([[]]), source=SourceArray([[]]), sym_load=SymLoadArray([[]]), sym_gen=SymGenArray([[]]), transformer_tap_regulator=TransformerTapRegulatorArray([[]]), sym_power_sensor=SymPowerSensorArray([[]]), sym_voltage_sensor=SymVoltageSensorArray([[]]), asym_voltage_sensor=AsymVoltageSensorArray([[]]))